In [1]:
from dotenv import load_dotenv
import os
import tweepy
import time

In [2]:
# Load environment variables from .env file
load_dotenv()

True

In [3]:
# Fetch the bearer token from the environment variables (required for v2)
bearer_token = os.getenv('BEARER_TOKEN')

In [4]:
try:
    client = tweepy.Client(bearer_token=bearer_token)
    print("Authentication successful.")
except tweepy.TweepyException as e:
    print(f"Error during authentication: {e}")
    exit()

Authentication successful.


In [5]:
# Define the search query
query = 'Artificial Intelligence'
# query = input("Enter Keywords for search query")

In [6]:
# Fetch tweets using API v2
try:
    response = client.search_recent_tweets(query=query, max_results=10, tweet_fields=['text', 'created_at'])
    
    # Check if data is returned
    if response.data:
        tweets = response.data  # Store the fetched tweets
        print(f"Fetched {len(tweets)} tweets.")
    else:
        print("No tweets found for this query.")
except tweepy.TweepyException as e:
    print(f"Error fetching tweets: {e}")

    # If rate-limited, inspect response headers for rate limit reset time
    if hasattr(e, 'response') and 'x-rate-limit-reset' in e.response.headers:
        reset_time = int(e.response.headers['x-rate-limit-reset'])
        current_time = int(time.time())
        wait_time = reset_time - current_time + 5 # Adding a 5 seconds buffer
        print(f"Rate limit exceeded. Try after {wait_time} seconds.")
        time.sleep(wait_time)  # Sleep until rate limit resets
    else:
        print("Unhandled error. Please check API access or query.")


Fetched 10 tweets.


In [7]:
print(tweets)

[<Tweet id=1856927494047240488 text='@DeSi__kaTTa AI-Powered Art is an innovative fusion of technology and creativity, where artificial intelligence algorithms assist in generating, enhancing, or even creating original pieces of artwork.'>, <Tweet id=1856927484962300385 text='@DeSi__kaTTa AI-powered art encompasses a range of artistic practices where artificial intelligence technologies are used to create, enhance, or inspire visual artworks'>, <Tweet id=1856927374119510412 text='A decentralized delivery platform powered by blockchain and artificial intelligence has the potential to revolutionize the sector by fostering a more just and efficient ecosystem. https://t.co/le55GcJPWW'>, <Tweet id=1856927358642479570 text='RT @andrewrsorkin: Ok. So this happened. \n\nIt is NOT us at all. \n\nFully generated by artificial intelligence. \n\nThe video, audio. All of it…'>, <Tweet id=1856927350576853387 text='I can\'t wait for AI to really become artificial intelligence so when people ask bulls

### 3.Preproccessing and cleaning the extracted tweets

In [8]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from autocorrect import Speller
from langdetect import detect
from nltk.corpus import words

In [9]:

# Download the required resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('words')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/divyagoyal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/divyagoyal/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/divyagoyal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/divyagoyal/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [10]:
# Function to preprocess tweet
spell = Speller()
def preprocess_tweet(tweet):
    # Remove Retweets (RT)
    tweet = re.sub(r'\brt\b', '', tweet)

    # Remove mentions (e.g., @username)
    tweet = re.sub(r'@\w+', '', tweet)

    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+', '', tweet)

    # Remove special characters, numbers, and punctuation
    tweet = re.sub(r'[^A-Za-z\s]', '', tweet)

    # Correct spelling mistakes using spell checker
    tweet = ' '.join([spell(word) for word in tweet.split()])

    # Convert text to lowercase
    tweet = tweet.lower()

    # Tokenize the tweet
    tokens = word_tokenize(tweet)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    return tokens

In [11]:
# Fetch and preprocess tweets
cleaned_tweets = []


In [12]:

for tweet in tweets:
    # Clean each tweet and append to the list
    cleaned_tweet = preprocess_tweet(tweet.text)
    cleaned_tweets.append(cleaned_tweet)

In [13]:
# Print cleaned and tokenized tweets
for tweet in cleaned_tweets:
    print(' '.join(tweet))

powered art innovative fusion technology creativity artificial intelligence algorithms assist generating enhancing even creating original pieces artwork
powered art encompasses range artistic practices artificial intelligence technologies used create enhance inspire visual artworks
decentralized delivery platform powered blockchain artificial intelligence potential revolutionized sector fostering efficient ecosystem
ok happened us fully generated artificial intelligence video audio
cant wait really become artificial intelligence people ask bullshit prompts like ai like fuck say
artificial amp biological intelligence visiting earth uaphearing
europe safe trustworthy humancentric inviting feedback prepare new guidelines help
focusing pin see bitcoin role evolving artificial intelligence applications particularly terms computation data storage
artificial intelligence
many users social media including x artificial intelligence computer bots tch ut influences lin donotletaifoolyou god expec

### 4. Performing Sentiment Analysis: Using TextBlob

In [14]:
pip install textblob



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
from textblob import TextBlob

In [16]:
# Function to get sentiment
def get_sentiment(text):
    # Create a TextBlob object
    blob = TextBlob(text)
    
    # Get the polarity (-1 to 1) and subjectivity (0 to 1)
    sentiment = blob.sentiment
    return sentiment.polarity, sentiment.subjectivity


In [17]:
# Perform sentiment analysis on cleaned tweets
for tweet in cleaned_tweets:
    # Join tokens back to form tweet text
    tweet_text = ' '.join(tweet)
    

In [18]:
# Get sentiment
polarity, subjectivity = get_sentiment(tweet_text)

In [19]:
 # Print the results
print(f"Tweet: {tweet_text}")
print(f"Polarity: {polarity}, Subjectivity: {subjectivity}\n")

Tweet: many users social media including x artificial intelligence computer bots tch ut influences lin donotletaifoolyou god expect us blind faith
Polarity: -0.14166666666666666, Subjectivity: 0.5583333333333333

